##MLflow quickstart part 2: serving models with Microsoft Azure ML

This notebook is part of a quickstart guide based on the [MLflow tutorial](https://www.mlflow.org/docs/latest/tutorial.html).
The [first part of the guide](https://docs.azuredatabricks.net/applications/mlflow/mlflow-training.html), **MLflow quickstart: model training and logging**, focuses on training a model and logging the training metrics, parameters, and model to the MLflow tracking server. 

##### NOTE: We do not recommend using *Run All* because it takes several minutes to deploy and update models; models cannot be queried until they are active.

This part of the guide consists of the following sections:

#### Setup
* Launch an Azure Databricks cluster
* Install MLflow
* Install the Azure ML SDK
* Create or load an Azure ML Workspace

#### Building an Azure Container Image for model deployment
* Use MLflow to build a Container Image for the trained model

#### Deploying the model to "dev" using Azure Container Instances (ACI)
* Create an ACI webservice deployment using the model's Container Image

#### Querying the deployed model in "dev"
* Load a sample input vector from the diabetes dataset
* Evaluate the sample input vector by sending an HTTP request

#### Deploying the model to production using Azure Kubernetes Service (AKS)
* Option 1: Create a new AKS cluster
* Option 2: Connect to an existing AKS cluster
* Deploy to the model's image to the specified AKS cluster

#### Querying the deployed model in production
* Load a sample input vector from the wine dataset
* Evaluate the sample input vector by sending an HTTP request

#### Updating the production deployment
* Build an Azure Container Image for another model
* Deploy the new model's image to the AKS cluster
* Query the updated model

#### Cleaning up the deployments
* Terminate the "dev" ACI webservice
* Terminate the production AKS webservice
* Remove the AKS cluster from the Azure ML Workspace

As in the first part of the quickstart tutorial, this notebook uses ElasticNet models trained on the `diabetes` dataset in scikit-learn.

**Note:** This notebook expects that you use an Azure Databricks hosted MLflow tracking server. To set up your own tracking server, see the instructions in [MLflow Tracking Servers](https://www.mlflow.org/docs/latest/tracking.html#mlflow-tracking-servers) and configure your connection to your tracking server by running [mlflow.set_tracking_uri](https://www.mlflow.org/docs/latest/python_api/mlflow.html#mlflow.set_tracking_uri).

## Prerequisites
ElasticNet models from the MLflow quickstart notebook in [part 1 of the quickstart guide](https://docs.azuredatabricks.net/applications/mlflow/mlflow-training.html).

## Setup

1. Ensure you are using or create a cluster specifying Python 3.
1. If you are running Databricks Runtime, uncomment and run Cmd 6 to install the required libraries. If you are running Databricks Runtime for Machine Learning, you can skip this step as the required libraries are already installed. 
1. Attach this notebook to the cluster.

In [0]:
#dbutils.library.installPyPI("mlflow", extras="extras")
#dbutils.library.installPyPI("azureml-sdk[databricks]")
#dbutils.library.restartPython()

### Create or load an Azure ML Workspace

Before models can be deployed to Azure ML, you must create or obtain an Azure ML Workspace. The `azureml.core.Workspace.create()` function will load a workspace of a specified name or create one if it does not already exist. For more information about creating an Azure ML Workspace, see the [Azure ML Workspace management documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-workspace).

In [0]:
#pip install azureml.core

In [0]:
import azureml
from azureml.core import Workspace

workspace_name = "MLFLow-DataD"
workspace_location="East US"
resource_group = "MLFlow-Databricks"
subscription_id = "1610f1b6-abaf-4f82-bee5-821c5f4786c2"

workspace = Workspace.create(name = workspace_name,
                             location = workspace_location,
                             resource_group = resource_group,
                             subscription_id = subscription_id,
                             exist_ok=True)

## Build an Azure Container Image for model deployment

### Use MLflow to build a Container Image for the trained model

Use the `mlflow.azuereml.build_image` function to build an Azure Container Image for the trained MLflow model. This function also registers the MLflow model with a specified Azure ML workspace. The resulting image can be deployed to Azure Container Instances (ACI) or Azure Kubernetes Service (AKS) for real-time serving.

Specify the run ID associated with an ElasticNet training run from [part 1 of the quickstart guide](https://docs.azuredatabricks.net/spark/latest/mllib/mlflow-tracking.html). You can find a run ID and model path from the experiment run, which can be found on the run details page:

![image](https://docs.azuredatabricks.net/_static/images/mlflow/mlflow-deployment-example-run-info.png)

In [0]:
run_id1 = "186f11aa17cb4af78ec68b6b8df2871a"
model_uri = "runs:/" + run_id1 + "/model"

In [0]:
#pip install mlflow

In [0]:
import mlflow.azureml

model_image, azure_model = mlflow.azureml.build_image(model_uri=model_uri, 
                                                      workspace=workspace,
                                                      model_name="model",
                                                      image_name="model",
                                                      description="Sklearn ElasticNet image for predicting diabetes progression",
                                                      synchronous=False)

<command-4138666497034493>:3: DeprecationWarning: ``mlflow.azureml.build_image`` is deprecated since 1.19.0. This method will be removed in a near future release. Use ``the azureml deployment plugin, https://aka.ms/aml-mlflow-deploy`` instead.
 model_image, azure_model = mlflow.azureml.build_image(model_uri=model_uri,
Registering model model
2022/02/20 10:40:04 INFO mlflow.azureml: Registered an Azure Model with name: `model` and version: `1`
Creating image
2022/02/20 10:40:18 INFO mlflow.azureml: Building an Azure Container Image with name: `model` and version: `1`

In [0]:
model_image.wait_for_creation(show_output=True)

Running............................................
Succeeded
Image creation operation finished for image model:1, operation "Succeeded"

## Deploy the model to "dev" using [Azure Container Instances (ACI)](https://docs.microsoft.com/en-us/azure/container-instances/)

The [ACI platform](https://docs.microsoft.com/en-us/azure/container-instances/) is the recommended environment for staging and developmental model deployments.

### Create an ACI webservice deployment using the model's Container Image

Using the Azure ML SDK, deploy the Container Image for the trained MLflow model to ACI.

In [0]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig, Model

dev_webservice_name = "diabetes-model1"
dev_webservice_deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
#dev_webservice = Webservice.deploy_from_image(name=dev_webservice_name, image=model_image, deployment_config=dev_webservice_deployment_config, workspace=workspace)

In [0]:
service = Model.deploy(
    workspace = workspace,
    name = "dev_webservice_name",
    models = "model",
   # inference_config = inference_config,
    deployment_config = dev_webservice_deployment_config)

--------------------------------------------------------------------------- 
 WebserviceException Traceback (most recent call last)
 <command-3110181578440722> in <module> 
 ----> 1 service = Model.deploy(
 2 workspace = workspace , 
 3 name = "dev_webservice_name" , 
 4 models = "model" , 
 5 # inference_config = inference_config, 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-b5be28d6-9b0c-4b92-a85e-e24ae737bccf/lib/python3.8/site-packages/azureml/core/model.py in deploy (workspace, name, models, inference_config, deployment_config, deployment_target, overwrite, show_output) 
 1635 from azureml . core . webservice . local import LocalWebserviceDeploymentConfiguration
 1636 
 -> 1637 webservice_name_validation ( name ) 
 1638 
 1639 if show_output : 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-b5be28d6-9b0c-4b92-a85e-e24ae737bccf/lib/python3.8/site-packages/azureml/_model_management/_util.py in webservice_name_validation (name) 
 1160 """
 1161 if not re . fullmatch ( r'^[a-z]([-a-z0-9]*[a-z0-9])?$' , name ) or len ( name ) < 3 or len ( name ) > 32 : 
 -> 1162 raise WebserviceException('Error, provided service name is invalid. It must only consist of lowercase '
 1163 'letters, numbers, or dashes, start with a letter, end with a letter or number, ' 
 1164 'and be between 3 and 32 characters long.')

 WebserviceException : WebserviceException:
	Message: Error, provided service name is invalid. It must only consist of lowercase letters, numbers, or dashes, start with a letter, end with a letter or number, and be between 3 and 32 characters long.
	InnerException None
	ErrorResponse 
{
 "error": {
 "message": "Error, provided service name is invalid. It must only consist of lowercase letters, numbers, or dashes, start with a letter, end with a letter or number, and be between 3 and 32 characters long."
 }
}

In [0]:
dev_webservice.wait_for_deployment()

Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 9ad733fb-43aa-4cc6-9801-4b5b67816db1
More information can be found using '.get_logs()'
Error:
{
 "code": "AciDeploymentFailed",
 "statusCode": 400,
 "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: diabetes-model1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image ee19418a463c4822b44e78af32288610.azurecr.io/model@sha256:321ace914d71e64532bfefbcac4e7802f52bbc89d902530533ac69cd8558fb24 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
 "details": [
 {
 "code": "CrashLoopBackOff",
 "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: diabetes-model1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image ee19418a463c4822b44e78af32288610.azurecr.io/model@sha256:321ace914d71e64532bfefbcac4e7802f52bbc89d902530533ac69cd8558fb24 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information."
 },
 {
 "code": "AciDeploymentFailed",
 "message": "Your container application crashed. Please follow the steps to debug:
	1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. Please refer to https://aka.ms/debugimage#dockerlog for more information.
	2. If your container application crashed. This may be caused by errors in your scoring file's init() function. You can try debugging locally first. Please refer to https://aka.ms/debugimage#debug-locally for more information.
	3. You can also interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	4. View the diagnostic events to check status of container, it may help you to debug the issue.
"RestartCount": 3
"CurrentState": {"state":"Waiting","startTime":null,"exitCode":null,"finishTime":null,"detailStatus":"CrashLoopBackOff: Back-off restarting failed"}
"PreviousState": {"state":"Terminated","startTime":"2022-02-20T13:04:42.999Z","exitCode":111,"finishTime":"2022-02-20T13:03:27.092Z","detailStatus":"Error"}
"Events": null
"
 }
 ]
}

--------------------------------------------------------------------------- 
 WebserviceException Traceback (most recent call last)
 <command-4138666497034498> in <module> 
 ----> 1 dev_webservice . wait_for_deployment ( ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-b5be28d6-9b0c-4b92-a85e-e24ae737bccf/lib/python3.8/site-packages/azureml/core/webservice/webservice.py in wait_for_deployment (self, show_output, timeout_sec) 
 917 logs_response = 'Current sub-operation type not known, more logs unavailable.' 
 918 
 --> 919 raise WebserviceException('Service deployment polling reached non-successful terminal state, current '
 920 'service state: {}\n' 
 921 'Operation ID: {}\n' 

 WebserviceException : WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 9ad733fb-43aa-4cc6-9801-4b5b67816db1
More information can be found using '.get_logs()'
Error:
{
 "code": "AciDeploymentFailed",
 "statusCode": 400,
 "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: diabetes-model1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image ee19418a463c4822b44e78af32288610.azurecr.io/model@sha256:321ace914d71e64532bfefbcac4e7802f52bbc89d902530533ac69cd8558fb24 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
 "details": [
 {
 "code": "CrashLoopBackOff",
 "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: diabetes-model1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image ee19418a463c4822b44e78af32288610.azurecr.io/model@sha256:321ace914d71e64532bfefbcac4e7802f52bbc89d902530533ac69cd8558fb24 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information."
 },
 {
 "code": "AciDeploymentFailed",
 "message": "Your container application crashed. Please follow the steps to debug:
	1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. Please refer to https://aka.ms/debugimage#dockerlog for more information.
	2. If your container application crashed. This may be caused by errors in your scoring file's init() function. You can try debugging locally first. Please refer to https://aka.ms/debugimage#debug-locally for more information.
	3. You can also interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	4. View the diagnostic events to check status of container, it may help you to debug the issue.
"RestartCount": 3
"CurrentState": {"state":"Waiting","startTime":null,"exitCode":null,"finishTime":null,"detailStatus":"CrashLoopBackOff: Back-off restarting failed"}
"PreviousState": {"state":"Terminated","startTime":"2022-02-20T13:04:42.999Z","exitCode":111,"finishTime":"2022-02-20T13:03:27.092Z","detailStatus":"Error"}
"Events": null
"
 }
 ]
}
	InnerException None
	ErrorResponse 
{
 "error": {
 "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 9ad733fb-43aa-4cc6-9801-4b5b6781

## Query the deployed model in "dev"

### Load diabetes dataset

In [0]:
from sklearn import datasets
diabetes = datasets.load_diabetes()

## Create sample input vector

In [0]:
import pandas as pd
import numpy as np

X = diabetes.data
y = diabetes.target
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)
sample = data.drop(["progression"], axis=1).iloc[[0]]
                                                 
query_input = sample.to_json(orient='split')
query_input = eval(query_input)
query_input.pop('index', None)

Out[11]: [0]

#### Evaluate the sample input vector by sending an HTTP request
Query the ACI webservice's scoring endpoint by sending an HTTP POST request that contains the input vector.

In [0]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [0]:
dev_webservice.scoring_uri

In [0]:
dev_prediction = query_endpoint_example(scoring_uri=dev_webservice.scoring_uri, inputs=query_input)

Sending batch prediction request with inputs: {'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], 'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]]}

--------------------------------------------------------------------------- 
 MissingSchema Traceback (most recent call last)
 <command-4138666497034507> in <module> 
 ----> 1 dev_prediction = query_endpoint_example ( scoring_uri = dev_webservice . scoring_uri , inputs = query_input ) 

 <command-4138666497034505> in query_endpoint_example (scoring_uri, inputs, service_key) 
 10 
 11 print ( "Sending batch prediction request with inputs: {}" . format ( inputs ) ) 
 ---> 12 response = requests . post ( scoring_uri , data = json . dumps ( inputs ) , headers = headers ) 
 13 preds = json . loads ( response . text ) 
 14 print ( "Received response: {}" . format ( preds ) ) 

 /databricks/python/lib/python3.8/site-packages/requests/api.py in post (url, data, json, **kwargs) 
 117 """
 118 
 --> 119 return request ( 'post' , url , data = data , json = json , ** kwargs ) 
 120 
 121 

 /databricks/python/lib/python3.8/site-packages/requests/api.py in request (method, url, **kwargs) 
 59 # cases, and look like a memory leak in others. 
 60 with sessions . Session ( ) as session : 
 ---> 61 return session . request ( method = method , url = url , ** kwargs ) 
 62 
 63 

 /databricks/python/lib/python3.8/site-packages/requests/sessions.py in request (self, method, url, params, data, headers, cookies, files, auth, timeout, allow_redirects, proxies, hooks, stream, verify, cert, json) 
 526 hooks = hooks , 
 527 )
 --> 528 prep = self . prepare_request ( req ) 
 529 
 530 proxies = proxies or { } 

 /databricks/python/lib/python3.8/site-packages/requests/sessions.py in prepare_request (self, request) 
 454 
 455 p = PreparedRequest ( ) 
 --> 456 p.prepare(
 457 method = request . method . upper ( ) , 
 458 url = request . url , 

 /databricks/python/lib/python3.8/site-packages/requests/models.py in prepare (self, method, url, headers, files, data, params, auth, cookies, hooks, json) 
 314 
 315 self . prepare_method ( method ) 
 --> 316 self . prepare_url ( url , params ) 
 317 self . prepare_headers ( headers ) 
 318 self . prepare_cookies ( cookies ) 

 /databricks/python/lib/python3.8/site-packages/requests/models.py in prepare_url (self, url, params) 
 388 error = error . format ( to_native_string ( url , 'utf8' ) ) 
 389 
 --> 390 raise MissingSchema ( error ) 
 391 
 392 if not host : 

 MissingSchema : Invalid URL 'None': No schema supplied. Perhaps you meant http://None?

## Deploy the model to production using [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/en-us/services/kubernetes-service/). Do Option 1 or Option 2.

### Option 1: Create a new AKS cluster

If you do not have an active AKS cluster for model deployment, create one using the Azure ML SDK.

In [0]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_cluster_name = "diabetes-cluster" 
# Create the cluster
aks_target = ComputeTarget.create(workspace = workspace, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating.........................................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None

### Option 2: Connect to an existing AKS cluster

If you already have an active AKS cluster running, you can add it to your Workspace using the Azure ML SDK.

In [0]:
from azureml.core.compute import AksCompute, ComputeTarget

# Get the resource group from https://porta..azure.com -> Find your resource group
resource_group = "<resource-group>"

# Give the cluster a local name
aks_cluster_name = "diabetes-cluster"

# Attatch the cluster to your workgroup
attach_config = AksCompute.attach_configuration(resource_group=resource_group, cluster_name=aks_cluster_name)
aks_target = ComputeTarget.attach(workspace, name="diabetes-compute", attach_config)

# Wait for the operation to complete
aks_target.wait_for_completion(True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

### Deploy to the model's image to the specified AKS cluster

In [0]:
from azureml.core.webservice import Webservice, AksWebservice

# Set configuration and service name
prod_webservice_name = "diabetes-model-prod"
prod_webservice_deployment_config = AksWebservice.deploy_configuration()

# Deploy from image
prod_webservice = Webservice.deploy_from_image(workspace = workspace, 
                                               name = prod_webservice_name,
                                               image = model_image,
                                               deployment_config = prod_webservice_deployment_config,
                                               deployment_target = aks_target)

Creating service

In [0]:
# Wait for the deployment to complete
prod_webservice.wait_for_deployment(show_output = True)

Running..............
SucceededAKS service creation operation finished, operation "Succeeded"

## Query the deployed model in production

#### Evaluate the sample input vector by sending an HTTP request
Query the AKS webservice's scoring endpoint by sending an HTTP POST request that includes the input vector. The production AKS deployment may require an authorization token (service key) for queries. Include this key in the HTTP request header.

In [0]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [0]:
prod_scoring_uri = prod_webservice.scoring_uri
prod_service_key = prod_webservice.get_keys()[0] if len(prod_webservice.get_keys()) > 0 else None

In [0]:
prod_prediction1 = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input)

Sending batch prediction request with inputs: {'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], 'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]]}
Received response: [209.994170046977]

## Update the production deployment

### Build an Azure Container Image for the new model

In [0]:
run_id2 = "<run-id2>"
model_uri = "runs:/" + run_id2 + "/model"

In [0]:
import mlflow.azureml

model_image_updated, azure_model_updated = mlflow.azureml.build_image(model_uri=model_uri, 
                                                                      workspace=workspace,
                                                                      model_name="model-updated",
                                                                      image_name="model-updated",
                                                                      description="Sklearn ElasticNet image for predicting diabetes progression",
                                                                      synchronous=False)

Registering model model-updated
Creating image

In [0]:
model_image_updated.wait_for_creation(show_output=True)

Running...........................................................
Succeeded
Image creation operation finished for image model-updated:2, operation "Succeeded"

### Deploy the new model's image to the AKS cluster

Using the [`azureml.core.webservice.AksWebservice.update()`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py#update) function, replace the deployment's existing model image with the new model image.

In [0]:
prod_webservice.update(image=model_image_updated)

In [0]:
prod_webservice.wait_for_deployment(show_output = True)

Running..................
SucceededAKS service creation operation finished, operation "Succeeded"

### Query the updated model

In [0]:
prod_prediction2 = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input)

Sending batch prediction request with inputs: {'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], 'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]]}
Received response: [172.73084996146514]

## Compare the predictions

In [0]:
print("Run ID: {} Prediction: {}".format(run_id1, prod_prediction1)) 
print("Run ID: {} Prediction: {}".format(run_id2, prod_prediction2))

Run ID: 434346c76d62407db8712075feb0ebb0 Prediction: [209.994170046977]
Run ID: fe735e9949ca484c8c090d9ffcd88272 Prediction: [172.73084996146514]

## Clean up the deployments

### Terminate the "dev" ACI webservice

Because ACI manages compute resources on your behalf, deleting the "dev" ACI webservice will remove all resources associated with the "dev" model deployment

In [0]:
dev_webservice.delete()

### Terminate the production AKS webservice

This terminates the real-time serving webservice running on the specified AKS cluster. It **does not** terminate the AKS cluster.

In [0]:
prod_webservice.delete()

### Remove the AKS cluster from the Azure ML Workspace

If the cluster was created using the Azure ML SDK (see **Option 1: Create a new AKS cluster**), remove it from the Azure ML Workspace will terminate the cluster, including all of its compute resources and deployments.

If the cluster was created independently (see **Option 2: Connect to an existing AKS cluster**), it will remain active after removal from the Azure ML Workspace.

In [0]:
aks_target.delete()